<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/OR_Tools_for_Routing_Problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ortools

In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def create_data_model():
    return {
        'distance_matrix': [
            [0, 2, 9, 10],
            [1, 0, 6, 4],
            [15, 7, 0, 8],
            [6, 3, 12, 0],
        ],
        'num_vehicles': 1,
        'depot': 0
    }

def print_solution(manager, routing, solution):
    index = routing.Start(0)
    plan_output = 'Route:\n'
    while not routing.IsEnd(index):
        plan_output += f'{manager.IndexToNode(index)} -> '
        index = solution.Value(routing.NextVar(index))
    plan_output += f'{manager.IndexToNode(index)}\n'
    print(plan_output)

data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)